In [1]:
import pyqg
import numpy as np
import matplotlib.pyplot as plt

# Run four models -- [2,3] layers x [64, 96] resolution

In [2]:
year = 24*60*60*360.

models = [[],[]]

for i, model_class in enumerate([pyqg.LayeredModel, pyqg.QGModel]):
    for nx in [96,64]:
        model = model_class(nx=nx, tmax=10*year, twrite=10000, tavestart=5*year, dt=3600)
        model.run()
        models[i].append(model)

INFO:  Logger initialized
INFO: Step: 10000, Time: 3.60e+07, KE: 8.41e-03, CFL: 0.247
INFO: Step: 20000, Time: 7.20e+07, KE: 1.66e-02, CFL: 0.322
INFO: Step: 30000, Time: 1.08e+08, KE: 1.71e-02, CFL: 0.334
INFO: Step: 40000, Time: 1.44e+08, KE: 1.94e-02, CFL: 0.326
INFO: Step: 50000, Time: 1.80e+08, KE: 1.42e-02, CFL: 0.331
INFO: Step: 60000, Time: 2.16e+08, KE: 1.57e-02, CFL: 0.329
INFO: Step: 70000, Time: 2.52e+08, KE: 1.73e-02, CFL: 0.311
INFO: Step: 80000, Time: 2.88e+08, KE: 1.60e-02, CFL: 0.317
INFO:  Logger initialized
INFO: Step: 10000, Time: 3.60e+07, KE: 7.50e-03, CFL: 0.130
INFO: Step: 20000, Time: 7.20e+07, KE: 1.37e-02, CFL: 0.194
INFO: Step: 30000, Time: 1.08e+08, KE: 1.31e-02, CFL: 0.177
INFO: Step: 40000, Time: 1.44e+08, KE: 9.65e-03, CFL: 0.185
INFO: Step: 50000, Time: 1.80e+08, KE: 1.21e-02, CFL: 0.176
INFO: Step: 60000, Time: 2.16e+08, KE: 1.17e-02, CFL: 0.182
INFO: Step: 70000, Time: 2.52e+08, KE: 1.37e-02, CFL: 0.181
INFO: Step: 80000, Time: 2.88e+08, KE: 1.18e-02,

# Check diagnostics for proper behavior / normalization

For all diagnostics, the lower-resolution model should have smaller extremes. However, lower-resolution model diagnostics should still have the same order of magnitude.

In [6]:
for models_by_res in models:
    print(models_by_res[0].__class__.__name__)
    diagnostics = models_by_res[0].diagnostics.keys()
    for diagnostic in diagnostics:
        print('  ', diagnostic)
        vals = []
        for m in models_by_res:
            max_val = np.abs(m.get_diagnostic(diagnostic)).max()
            print(f"      nx={m.nx}: {max_val}")
            vals.append(max_val)
        max_hi, max_lo = vals 
        assert max_lo <= max_hi
        if max_lo > 0:
            assert max_hi/max_lo < 10

LayeredModel
   Ensspec
      nx=96: 2.7788592123816947e-11
      nx=64: 2.2846493542927073e-11
   KEspec
      nx=96: 0.0008382496483329544
      nx=64: 0.0006658234122507195
   EKEdiss
      nx=96: 4.457300980209053e-09
      nx=64: 3.409288403143049e-09
   KEfrictionspec
      nx=96: 8.229379313824047e-11
      nx=64: 6.565242610200721e-11
   EKE
      nx=96: 0.045355616558633405
      nx=64: 0.03615316073365993
   Dissspec
      nx=96: 6.506728019383517e-05
      nx=64: 5.502307651468687e-05
   paramspec
      nx=96: 0.0
      nx=64: 0.0
   ENSDissspec
      nx=96: 3.735920497296309e-12
      nx=64: 1.5765957271888285e-12
   entspec
      nx=96: 6.071727587662209e-10
      nx=64: 5.222706425948921e-10
   KEspec_modal
      nx=96: 0.0004486228018525332
      nx=64: 0.0003619015205304205
   PEspec_modal
      nx=96: 0.00047687665813923695
      nx=64: 0.00037249214871873653
   APEspec
      nx=96: 0.0005829909158195586
      nx=64: 0.00046108722325958654
   KEflux_div
      nx=96: 9.